## Imports

In [ ]:
# I am going to implement a driver for Keil UVision 5 for a ARM Cortex-M4 LPC4078 microcontroller. The driver will be implemented in Python 3.6.5 and then converted to C for the final implementation. The driver is concered with persistant memory storage to manipulate EEPROM control register as necessary to write and read to it. The memory needed to be stored are flags for an LCD display settings.

# Begin with the necessary imports
import os
import sys
import time
import random
import struct
import serial





## Driver Working Description

In [ ]:
#EEPROM device description
#EEPROM is a non-volatile memory mainly used for storing relatively small amounts of data, for example for storing settings. There are three operations for accessing the memory: reading, writing and erase/program. "Writing" to memory is split up into two separate operations, writing and erase/program. The first operation, which will be called "writing" in this document, is not really updating the memory, but only updating the temporary data register called the "page register". The page register needs to be written with a minimum 1 byte and a maximum 64 bytes before the second operation, which is called "erase/program" in this document, can be used to actually update the non-volatile memory. Note that the data written to the page register is not "cached"; it cannot be read before it is actually programmed into non-volatile memory.

#The 64-byte page register is the same size as a page in EEPROM memory. The 4,032 bytes EEPROM on most devices contains 63 pages. Devices with a 2 kB EEPROM provide 2,048 bytes on 32 pages.37.4.2 EEPROM operations. An EEPROM device cannot be programmed directly. Writing data to it and the actual erase/program of the memory are two separate steps. The page register (64 bytes) will temporarily hold write data. But as soon as this data needs to be read from the EEPROM or data needs to be written to another page, the contents of the page register first needs to be programmed into the EEPROM memory. The following sections explain the EEPROM operations (read, write and erase/program) in more detail.

#37.4.2.1 Writing: The EEPROM controller supports writing of 8-bit, 16-bit, or 32-bit elements. Since the EEPROM device doesn’t directly support 32-bit write operations, the controller splits the operation into two 16-bit operations.For doing a write operation first an address needs to be written into the address register and the kind of write operation needs to be selected in the command register. This can be done in any order. After this the data is written to the write data register, which automatically starts the write operation on the EEPROM device. UM10562 All information provided in this document is subject to legal disclaimers. © NXP B.V. 2014. All rights reserved. User manual Rev. 3 — 12 March 2014 864 of 947cNXP Semiconductors UM10562 

#Chapter 37: LPC408x/407x EEPROM memory - A write operation causes an automatic post-increment of the address. This allows consecutive writes to the page register without the need of writing a new address for every write operation. Of course the address register could be written with another address value to write to another location. If the data register is written while a previous EEPROM operation is still pending, the write transfer on the system bus is stalled until the previous operation is finished. This can be avoided by polling the interrupt status register to see if an operation is still pending before starting the write operation. Software has to make sure that the following rules are followed: • overwriting (writing it two times before an erase/program operation) one of the locations in a 64-byte page register is not allowed. It will cause the loss of the previously written data 


#Chapter 37: LPC408x/407x EEPROM memory - • in case the default address post-incrementing is used, the upper boundary of the page register may not be crossed • the contents of the page register needs to be programmed into non-volatile memory before it can be read back • write operations to a misaligned address will result in an error response on the write transfer to the write data register (for example a 32-bit write operation to an address other than a multiple of 0x4). The operation will not be performed.

#37.4.2.2 Erase/Programming - After the page register has been written with user data, it still has to programmed into non-volatile memory. This is a separate step. Only writing to the page register will not write the EEPROM memory. Programming the page into memory takes a relatively long time, therefore the corresponding interrupt can be enabled, or the interrupt status bit can be polled to avoid stalling of the system bus. An erase/program operation starts by providing the MSBs of the address that selects the page in memory. The 6 LSBs are "don’t care". The operation is started by writing the command register (selecting the erase/program operation). Before beginning a programming operation, the EEPROM status should be polled to insure that the last write operation has been completed.

#37.4.2.3 Reading - The EEPROM controller supports reading 8-bit, 16-bit, or 32-bit elements. Since the EEPROM device doesn’t support 32-bit operations the controller splits the operation into two 16-bit operations. For doing a read operation, an address first needs to be written into the address register. Then the operation needs to be selected in the command register. Writing the command register will automatically start the read operation on the EEPROM device.

#Chapter 37: LPC408x/407x EEPROM memory - If the read data register is read while the read operation is still pending, then the read transfer on the system bus is stalled until the previous read operation is finished. This can be avoided by polling the interrupt status register to see if the operation is still pending before reading the read data register. Read operations will automatically post-increment the address register. This allows consecutive reads from the EEPROM memory without the need of writing a new address for every read operation. By setting the read data prefetch bit in the command register, reading from the read data register automatically starts up a read operation from the next (incremented) address location. When doing consecutive reads in this way, the first read operation is started as result of writing the command register. The following read operations are started as result of reading the read data register to obtain the result of the previous read operations.Read operations from a misaligned address will result on an error response on the write transfer to the command register (for example a 32-bit read operation from an address other than a multiple of 0x4). The operation will not be performed.

#37.4.2.4 Exceptions - The controller can generate exceptions in the following situations:• Writing a read-only register or reading a write-only register • A transfer to a non-existing register location

#EEPROM registers - CMD R/W 0x080 EEPROM command register, ADDR R/W 0x084 EEPROM address register, WDATA WO 0x088 EEPROM write data register, RDATA RO 0x08C EEPROM read data register, WSTATE R/W 0x090 EEPROM wait state register, CLKDIV R/W 0x094 EEPROM clock divider register, PWRDWN R/W 0x098 EEPROM power-down register.

#EEPROM interrupt registers: INTSTAT RO 0xFE0 EEPROM interrupt status, INTSTATCLR WO 0xFE8 EEPROM interrupt status clear, INTSTATSET WO 0xFEC EEPROM interrupt status set, INTEN RO 0xFE4 EEPROM interrupt enable, INTENCLR WO 0xFD8 EEPROM interrupt enable clear, INTENSET WO 0xFDC EEPROM interrupt enable set.



## Driver Implementaion

In [ ]:
# The EEPROM driver will be implemented as a class
class EEPROM:
    def __init__(self):
        self.CMD = 0x080
        self.ADDR = 0x084
        self.WDATA = 0x088
        self.RDATA = 0x08C
        self.WSTATE = 0x090
        self.CLKDIV = 0x094
        self.PWRDWN = 0x098
        self.INTSTAT = 0xFE0
        self.INTSTATCLR = 0xFE8
        self.INTSTATSET = 0xFEC
        self.INTEN = 0xFE4
        self.INTENCLR = 0xFD8
        self.INTENSET = 0xFDC
        self.eeprom = serial.Serial('/dev/ttyUSB0', 9600, timeout=1)
        self.eeprom.isOpen()
        self.eeprom.write(b'Hello')
        self.eeprom.close()

    def write(self, address, data):
        self.eeprom.open()
        self.eeprom.write(struct.pack('I', self.ADDR))
        self.eeprom.write(struct.pack('I', address))
        self.eeprom.write(struct.pack('I', self.WDATA))
        self.eeprom.write(struct.pack('I', data))
        self.eeprom.close()

    def read(self, address):
        self.eeprom.open()
        self.eeprom.write(struct.pack('I', self.ADDR))
        self.eeprom.write(struct.pack('I', address))
        self.eeprom.write(struct.pack('I', self.RDATA))
        data = self.eeprom.read(4)
        self.eeprom.close()
        return struct.unpack('I', data)[0]

    def erase(self, address):
        self.eeprom.open()
        self.eeprom.write(struct.pack('I', self.ADDR))
        self.eeprom.write(struct.pack('I', address))
        self.eeprom.write(struct.pack('I', self.WDATA))
        self.eeprom.write(struct.pack('I', 0xFFFFFFFF))
        self.eeprom.close()

    def program(self, address):
        self.eeprom.open()
        self.eeprom.write(struct.pack('I', self.ADDR))
        self.eeprom.write(struct.pack('I', address))
        self.eeprom.write(struct.pack('I', self.CMD))
        self.eeprom.write(struct.pack('I', 0x00000001))
        self.eeprom.close()

    def read_status(self):





/*------------- EEPROM Controller (EEPROM) -----------------------------------*/
typedef struct
{
  __IO uint32_t CMD;            /* 0x0080 */
  __IO uint32_t ADDR;
  __IO uint32_t WDATA;
  __IO uint32_t RDATA;
  __IO uint32_t WSTATE;         /* 0x0090 */
  __IO uint32_t CLKDIV;
  __IO uint32_t PWRDWN;         /* 0x0098 */
       uint32_t RESERVED0[975];
  __IO uint32_t INT_CLR_ENABLE; /* 0x0FD8 */
  __IO uint32_t INT_SET_ENABLE;
  __IO uint32_t INT_STATUS;     /* 0x0FE0 */
  __IO uint32_t INT_ENABLE;
  __IO uint32_t INT_CLR_STATUS;
  __IO uint32_t INT_SET_STATUS;
} LPC_EEPROM_TypeDef;


In [ ]:
#include "eeprom.h"

// Define the EEPROM size (256 bytes)
#define EEPROM_SIZE 256
// Define the EEPROM control register (address 0x00200080)
#define EEPROM_CTRL_REG 0x00200080
// Define the EEPROM address register (address 0x00200084)
#define EEPROM_ADDR_REG 0x00200084
// Define the EEPROM data register (address 0x00200088)
#define EEPROM_DATA_REG 0x00200088
// Define the EEPROM busy flag (bit 0)
#define EEPROM_BUSY 0x01
// Define the EEPROM write enable flag (bit 1)
#define EEPROM_WRITE_ENABLE 0x02
// Define the EEPROM read enable flag (bit 2)
#define EEPROM_READ_ENABLE 0x04
// Define the EEPROM erase enable flag (bit 3)
#define EEPROM_ERASE_ENABLE 0x08


// Function to write data to the EEPROM
void writeEEPROM(unsigned int address, unsigned char data) {
    // Check if the address is within the EEPROM range
    if (address >= EEPROM_BASE && address < EEPROM_BASE + EEPROM_SIZE) {
        // Set the address register to the desired address
        *(unsigned int *)EEPROM_ADDR_REG = address;
        // Set the data register to the desired data
        *(unsigned char *)EEPROM_DATA_REG = data;
        // Set the control register to write enable
        *(unsigned char *)EEPROM_CTRL_REG = EEPROM_WRITE_ENABLE;
        // Wait for the EEPROM to finish writing
        while (*(unsigned char *)EEPROM_CTRL_REG & EEPROM_BUSY);
    } else {
        // Print an error message if the address is out of range
        printf("Error: Address out of range\n");
    }
}

// Function to read data from the EEPROM
unsigned char readEEPROM(unsigned int address) {
    // Check if the address is within the EEPROM range
    if (address >= EEPROM_BASE && address < EEPROM_BASE + EEPROM_SIZE) {
        // Set the address register to the desired address
        *(unsigned int *)EEPROM_ADDR_REG = address;
        // Set the control register to read enable
        *(unsigned char *)EEPROM_CTRL_REG = EEPROM_READ_ENABLE;
        // Wait for the EEPROM to finish reading
        while (*(unsigned char *)EEPROM_CTRL_REG & EEPROM_BUSY);
        // Return the data read from the data register
        return *(unsigned char *)EEPROM_DATA_REG;
    } else {
        // Print an error message if the address is out of range
        printf("Error: Address out of range\n");
        // Return 0 if the address is out of range
        return 0;
    }
}

// Function to erase data from the EEPROM
void eraseEEPROM(unsigned int address) {
    // Check if the address is within the EEPROM range
    if (address >= EEPROM_BASE && address < EEPROM_BASE + EEPROM_SIZE) {
        // Set the address register to the desired address
        *(unsigned int *)EEPROM_ADDR_REG = address;
        // Set the control register to erase enable
        *(unsigned char *)EEPROM_CTRL_REG = EEPROM_ERASE_ENABLE;
        // Wait for the EEPROM to finish erasing
        while (*(unsigned char *)EEPROM_CTRL_REG & EEPROM_BUSY);
    } else {
        // Print an error message if the address is out of range
        printf("Error: Address out of range\n");
    }
}

// Function to print the contents of the EEPROM
void printEEPROM() {
    // Iterate over the EEPROM range
    for (unsigned int i = EEPROM_BASE; i < EEPROM_BASE + EEPROM_SIZE; i++) {
        // Read the data at the current address
        unsigned char data = readEEPROM(i);
        // Print the address and data
        printf("Address: 0x%04X, Data: 0x%02X\n", i, data);
    }
}

// Main function to test the EEPROM driver
int main() {
    // Write data to the EEPROM
    writeEEPROM(EEPROM_BASE, 0xAA);
    writeEEPROM(EEPROM_BASE + 1, 0xBB);
    writeEEPROM(EEPROM_BASE + 2, 0xCC);
    // Print the contents of the EEPROM
    printEEPROM();
    // Erase data from the EEPROM
    eraseEEPROM(EEPROM_BASE + 1);
    // Print the contents of the EEPROM
    printEEPROM();
    // Return 0 to indicate successful execution
    return 0;
}

In [ ]:
#include <stdio.h>
#include <stdlib.h>
#include <string.h>


// Define the EEPROM base address (base address 0x00200000)
#define EEPROM_BASE 0x00200000
// Define the EEPROM command register (CMD - address 0x00200080)
#define EEPROM_CMD 0x00200080
// Define the EEPROM address register (ADDR - address 0x00200084)
#define EEPROM_ADDR 0x00200084
// Define the EEPROM write data register (WDATA - address 0x00200088)
#define EEPROM_WDATA 0x00200088
// Define the EEPROM read data register (RDATA - address 0x0020008C)
#define EEPROM_RDATA 0x0020008C
// Define the EEPROM wait state register (WSTATE - address 0x00200090)
#define EEPROM_WSTATE 0x00200090
// Define the EEPROM clock divider register (CLKDIV - address 0x00200094)
#define EEPROM_CLKDIV 0x00200094
// Define the EEPROM power down/DCM register (PWRDWN - address 0x00200098) 
#define EEPROM_PWRDWN 0x00200098


void writeEEPROM(unsigned int address, unsigned char data);
unsigned char readEEPROM(unsigned int address);
void eraseEEPROM(unsigned int address);
void printEEPROM();
int main();

